# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
data_train.loc[data_train['quantity']>1000]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
5380,932,27115512702,12,6534178,1461,3.20,422,-0.01,1757,2,0.0,0.0
6213,778,27130432788,13,6534178,2416,5.00,310,-0.24,1838,3,0.0,0.0
6674,778,27130955980,14,6534178,2728,6.00,422,-0.27,1222,3,0.0,0.0
7198,778,27153895714,15,6534178,8737,20.00,310,-0.01,1147,3,0.0,0.0
8055,988,27176877230,17,6534178,11066,23.01,408,-1.10,1845,3,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2485323,1195,41931750869,684,6534178,9094,20.00,343,-0.01,1720,98,0.0,0.0
2485324,245,41931810998,684,1404121,11531,25.01,2974,-0.35,1336,98,0.0,0.0
2487660,1510,41944741332,684,6534178,6699,15.00,34280,-0.01,2238,98,0.0,0.0
2488799,275,41944892436,684,6534178,7146,15.00,313,-0.71,1518,98,0.0,0.0


In [4]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [6]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

C:\Users\Laptop\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Decreased # items from 90386 to 4944


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
# %%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py
4. Создайте модуль src.recommenders.py. Напищите код для класса ниже и положите его в src.recommenders.py
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [11]:
def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    popular_items = data_train.loc[data_train['user_id']== user].groupby('item_id')['quantity'].count().reset_index()
    popular_items.sort_values('quantity', ascending=False, inplace=True)
    popular_items = popular_items.head(N)
    user_items = [x for x in popular_items['item_id']]
    res = []
    for item in user_items:
        recs = model.similar_items(itemid_to_id[item], N=2)
        top_rec = recs[1][0]
        res.append(id_to_itemid[top_rec])
    return res

def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    res =[]
    if user in userid_to_id:
        for user, score in model.similar_users(userid_to_id[user], N=N+1)[1:]:
            popular_items = data_train.loc[data_train['user_id']== id_to_userid[user]].groupby('item_id')['quantity'].count().reset_index()
            popular_items.sort_values('quantity', ascending=False, inplace=True)
            user_item = popular_items.head(1)['item_id']
            res.append(user_item.item())
    
    return res

In [12]:
result['similar_items'] = result['user_id'].apply(lambda x: get_similar_items_recommendation(x, model = model, N=5))

In [13]:
result

,user_id,actual,similar_items
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[927783, 866211, 1082185, 1124432, 1098066]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1076875, 1092026, 960318, 1075979, 1133018]"
2,5,"[913077, 1118028, 1386668]","[942778, 1105301, 9526411, 833025, 7441210]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[866211, 951412, 904360, 845208, 948650]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[866211, 849843, 988791, 1015247, 1133018]"
...,...,...,...
1986,2494,"[826265, 828393, 834303, 835236, 839147, 84036...","[866211, 904360, 833025, 1075074, 889731]"
1987,2497,"[906777, 996833, 1032967, 15830980, 15928068, ...","[834484, 860776, 890283, 1125904, 819255]"
1988,2498,"[1014185, 13417441, 993638, 906335, 991948, 10...","[1126899, 866211, 942778, 1092026, 1127831]"
1989,2499,"[834117, 844441, 999971, 15924955, 18148096, 8...","[1126899, 1098066, 9526411, 845078, 905913]"


In [14]:
result['similar_users'] = result['user_id'].apply(lambda x: get_similar_users_recommendation(x, model = model, N=5))

In [15]:
result

,user_id,actual,similar_items,similar_users
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[927783, 866211, 1082185, 1124432, 1098066]","[1070820, 1082185, 904360, 1040781, 1029743]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1076875, 1092026, 960318, 1075979, 1133018]","[953476, 1037840, 1118393, 8090521, 1070820]"
2,5,"[913077, 1118028, 1386668]","[942778, 1105301, 9526411, 833025, 7441210]","[13511722, 1075368, 1058997, 1075368, 921345]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[866211, 951412, 904360, 845208, 948650]","[1024306, 995242, 1106523, 1032549, 1028816]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[866211, 849843, 988791, 1015247, 1133018]","[819330, 1029743, 1082185, 1082185, 1129822]"
...,...,...,...,...
1986,2494,"[826265, 828393, 834303, 835236, 839147, 84036...","[866211, 904360, 833025, 1075074, 889731]","[6424447, 8090536, 1126899, 1082185, 1126899]"
1987,2497,"[906777, 996833, 1032967, 15830980, 15928068, ...","[834484, 860776, 890283, 1125904, 819255]","[1037840, 1029743, 1082185, 873964, 1029743]"
1988,2498,"[1014185, 13417441, 993638, 906335, 991948, 10...","[1126899, 866211, 942778, 1092026, 1127831]","[1010190, 1082185, 1082185, 1426702, 947267]"
1989,2499,"[834117, 844441, 999971, 15924955, 18148096, 8...","[1126899, 1098066, 9526411, 845078, 905913]","[907631, 1029743, 1029743, 982790, 1082185]"


In [16]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.popularity.sort_values('quantity', ascending=False, inplace=True)
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, index='user_id', columns='item_id', values='quantity',
                                  aggfunc='count', fill_value=0)
        
        user_item_matrix = user_item_matrix.astype(float) 
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        res = []
        topn_items = self.popularity.loc[self.popularity['user_id']== user]
        topn_items = topn_items.head(N)
        user_items = [x for x in topn_items['item_id']]
        
        for item in user_items:
            recs = self.model.similar_items(itemid_to_id[item], N=2)
            top_rec = recs[1][0]
            res.append(id_to_itemid[top_rec])

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        res =[]
        if user in userid_to_id:
            for user, score in self.model.similar_users(userid_to_id[user], N=N+1)[1:]:
                user_items = self.popularity.loc[self.popularity['user_id']== id_to_userid[user]]
                top_item = top_items.head(1)['item_id']
                res.append(top_item.item())

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

In [17]:
def prefilter_items(data_train):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
    
    
    
    # Уберем самые популярные 
    
    # Уберем самые непопулряные 
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    recent = data_train.loc[(data_train['week_no'] >= data_train['week_no'].max() - 48) & (data_train['quantity'] != 0)]
    recent_items = recent.groupby('item_id').count().reset_index().item_id.to_list()
    data_train = data_train.loc[data_train['item_id'].isin(recent_items)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    prices = data_train.groupby('item_id')['sales_value'].max().reset_index()
    prices.sort_values('sales_value', ascending=False, inplace = True)
    cheap_items = prices.loc[prices.sales_value < 1]
    cheap_items = cheap_items.item_id.to_list()
    data_train = data_train.loc[~data_train['item_id'].isin(cheap_items)]    
    
    # Уберем слишком дорогие товары
    expensive_items = prices.loc[prices.sales_value > 100]
    expensive_items = expensive_items.item_id.to_list()
    data_train = data_train.loc[~data_train['item_id'].isin(expensive_items)]
    # ...
    
    return data_train
def postfilter_items():
    pass

Проверка, что все работает

In [18]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender